In [561]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

In [562]:
target = pd.read_parquet('target.parquet')


In [563]:
target = target[target['region'] == 'Москва'].reset_index().drop(columns = ['index'])
target.shape

(242421, 12)

In [564]:
target.columns

Index(['report_date', 'latitude', 'longitude', 'region', 'city', 'street',
       'house', 'finish', 'floor', 'rooms', 'area', 'price'],
      dtype='object')

In [565]:
X = target[['floor', 'rooms', 'area', 'price']]

In [566]:
X[['floor', 'full_floor']] = X.loc[:, ('floor')].str.split('/', expand = True)
X_1 = X[['floor', 'rooms', 'area', 'price']]

C:\Users\smyko\AppData\Local\Temp\ipykernel_2016\3655025119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['floor', 'full_floor']] = X.loc[:, ('floor')].str.split('/', expand = True)
C:\Users\smyko\AppData\Local\Temp\ipykernel_2016\3655025119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['floor', 'full_floor']] = X.loc[:, ('floor')].str.split('/', expand = True)


In [567]:
pd.unique(X_1['rooms'])

array(['2-комн. кв.', '2-комн. апарт.', '1-комн. кв.', '3-комн. кв.',
       '3-комн. апарт.', '1-комн. апарт.', '4-комн. кв.',
       '4-комн. апарт.', 'студия', 'апартаменты-студия', 'многокомн. кв.',
       '5-комн. кв.', '5-комн. апарт.', 'многокомн. апарт.', '6', '2',
       '3', '1', '5', '4'], dtype=object)

In [568]:
X_1['rooms'] = X_1['rooms'].replace({'2-комн. кв.': 2, 
                                     '3-комн. кв.': 3,
                                     '1-комн. кв.': 1,
                                     '4-комн. кв.': 4,
                                     'студия': 0.1,
                                     '2': 2,
                                     '3': 3,
                                     '1': 1,
                                     '4': 4,
                                    })
pd.unique(X_1['rooms'])

C:\Users\smyko\AppData\Local\Temp\ipykernel_2016\4057573386.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_1['rooms'] = X_1['rooms'].replace({'2-комн. кв.': 2,


array([2, '2-комн. апарт.', 1, 3, '3-комн. апарт.', '1-комн. апарт.', 4,
       '4-комн. апарт.', 0.1, 'апартаменты-студия', 'многокомн. кв.',
       '5-комн. кв.', '5-комн. апарт.', 'многокомн. апарт.', '6', '5'],
      dtype=object)

In [569]:
X_1.shape

(242421, 4)

In [570]:
X_2 = X_1[(X_1.rooms != '2-комн. апарт.') 
          & (X_1.rooms != '3-комн. апарт.')
          & (X_1.rooms != '1-комн. апарт.')
          & (X_1.rooms != 'апартаменты-студия')
          & (X_1.rooms != '4-комн. апарт.')
          & (X_1.rooms != '5-комн. апарт.')
          & (X_1.rooms != 'многокомн. апарт.')
          & (X_1.rooms != 'многокомн. кв.')
          & (X_1.rooms != '6')
          & (X_1.rooms != '5-комн. кв.')
          & (X_1.rooms != '5')]

In [571]:
X_2.shape

(231039, 4)

In [572]:
X_3 = X_2[(X_2.floor != '-2')
          & (X_2.floor != '-3')
          & (X_2.floor != '')]

In [573]:
#X_3 = X_3.loc[(X_3.rooms == 0.1) & (X_3.area <=32)]
#X_3 = X_3.query((X_3.rooms == 0.1) & (X_3.area <=32))
 
#X_3

In [574]:
conditions = [
    (X_3.rooms == 1) & (X_3.area < 32),
    (X_3.rooms == 1) & (X_3.area >= 32),
    (X_3.rooms == 2),
    (X_3.rooms == 3),
    (X_3.rooms == 4)
]

choices = ['0C', '1C', '2C', '3C', '4C']

X_3['type'] = np.select(conditions, choices)
X_3

C:\Users\smyko\AppData\Local\Temp\ipykernel_2016\929833784.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_3['type'] = np.select(conditions, choices)


,floor,rooms,area,price,type
0,6,2,51.000000,13400000.0,2C
1,2,2,42.000000,17900000.0,2C
2,5,2,42.000000,12500000.0,2C
3,1,2,42.000000,10990000.0,2C
4,4,2,51.000000,7450000.0,2C
...,...,...,...,...,...
242416,3,1,29.000000,15950000.0,0C
242417,10,2,89.599998,37500000.0,2C
242418,1,1,34.400002,10500000.0,1C
242419,12,3,105.400002,42000000.0,3C


In [575]:
conditions = [
    (X_3.type == '0C') & (X_3.area < 20),
    (X_3.type == '0C') & (X_3.area >= 20) & (X_3.area < 25),
    (X_3.type == '0C') & (X_3.area >= 25) & (X_3.area < 32),

    (X_3.type == '1C') & (X_3.area >= 32) & (X_3.area < 37),
    (X_3.type == '1C') & (X_3.area >= 37) & (X_3.area < 45),
    (X_3.type == '1C') & (X_3.area >= 45) & (X_3.area < 50),
    (X_3.type == '1C') & (X_3.area >= 50) & (X_3.area < 57),
    (X_3.type == '1C') & (X_3.area >= 57) & (X_3.area < 78),

    (X_3.type == '2C') & (X_3.area >= 34) & (X_3.area < 55),
    (X_3.type == '2C') & (X_3.area >= 55) & (X_3.area < 65),
    (X_3.type == '2C') & (X_3.area >= 65) & (X_3.area < 75),
    (X_3.type == '2C') & (X_3.area >= 75) & (X_3.area < 87),
    (X_3.type == '2C') & (X_3.area >= 87) & (X_3.area < 100),
    (X_3.type == '2C') & (X_3.area >= 100),

    (X_3.type == '3C') & (X_3.area >= 66) & (X_3.area < 75),
    (X_3.type == '3C') & (X_3.area >= 75) & (X_3.area < 85),
    (X_3.type == '3C') & (X_3.area >= 85) & (X_3.area < 100),
    (X_3.type == '3C') & (X_3.area >= 100) & (X_3.area < 115),
    (X_3.type == '3C') & (X_3.area >= 115) & (X_3.area < 135),
    (X_3.type == '3C') & (X_3.area >= 135),

    (X_3.type == '4C') & (X_3.area >= 150) & (X_3.area < 200),
    (X_3.type == '4C') & (X_3.area >= 200)
]

choices = ['XS', 'S', 'M', 
           'XS', 'S', 'M', 'L', 'XL',
           'XS', 'S', 'M', 'L', 'XL', 'oversize',
           'XS', 'S', 'M', 'L', 'XL', 'oversize',
           'XL', 'oversize']

X_3['flatsize'] = np.select(conditions, choices)
X_3

C:\Users\smyko\AppData\Local\Temp\ipykernel_2016\4223506609.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_3['flatsize'] = np.select(conditions, choices)


,floor,rooms,area,price,type,flatsize
0,6,2,51.000000,13400000.0,2C,XS
1,2,2,42.000000,17900000.0,2C,XS
2,5,2,42.000000,12500000.0,2C,XS
3,1,2,42.000000,10990000.0,2C,XS
4,4,2,51.000000,7450000.0,2C,XS
...,...,...,...,...,...,...
242416,3,1,29.000000,15950000.0,0C,M
242417,10,2,89.599998,37500000.0,2C,XL
242418,1,1,34.400002,10500000.0,1C,XS
242419,12,3,105.400002,42000000.0,3C,L


In [576]:
X_3_0 = X_3.loc[X_3.type == '0']
X_3_0

,floor,rooms,area,price,type,flatsize
30491,10,0.1,28.0,7500000.0,0,0
30492,1,0.1,28.0,4640560.0,0,0
30493,13,0.1,28.0,9820000.0,0,0
30495,5,0.1,28.0,6800000.0,0,0
30496,5,0.1,28.0,8400000.0,0,0
...,...,...,...,...,...,...
87372,17,0.1,25.0,6700000.0,0,0
87373,6,0.1,25.0,8500000.0,0,0
87375,18,0.1,25.0,8750000.0,0,0
87898,10,0.1,24.0,13000000.0,0,0


In [577]:
X_3 = X_3.loc[X_3.type != '0']


In [578]:
X_3

,floor,rooms,area,price,type,flatsize
0,6,2,51.000000,13400000.0,2C,XS
1,2,2,42.000000,17900000.0,2C,XS
2,5,2,42.000000,12500000.0,2C,XS
3,1,2,42.000000,10990000.0,2C,XS
4,4,2,51.000000,7450000.0,2C,XS
...,...,...,...,...,...,...
242416,3,1,29.000000,15950000.0,0C,M
242417,10,2,89.599998,37500000.0,2C,XL
242418,1,1,34.400002,10500000.0,1C,XS
242419,12,3,105.400002,42000000.0,3C,L


In [579]:
X_3['type'] = X_3['type'].replace({'0C': 0.1, 
                                     '1C': 1,
                                     '2C': 2,
                                     '3C': 3,
                                     '4C': 4})
pd.unique(X_3['type'])

C:\Users\smyko\AppData\Local\Temp\ipykernel_2016\890116648.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_3['type'] = X_3['type'].replace({'0C': 0.1,
C:\Users\smyko\AppData\Local\Temp\ipykernel_2016\890116648.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_3['type'] = X_3['type'].replace({'0C': 0.1,


array([2. , 1. , 3. , 0.1, 4. ])

In [580]:
X_3.shape

(228758, 6)

In [581]:
X_3['flatsize'] = X_3['flatsize'].replace({'XS': 1, 
                                   'S': 2,
                                   'M': 3,
                                   'L': 4,
                                   'XL': 5,
                                   'oversize':6})
pd.unique(X_3['flatsize'])

C:\Users\smyko\AppData\Local\Temp\ipykernel_2016\3069392814.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_3['flatsize'] = X_3['flatsize'].replace({'XS': 1,


array([1, 2, 3, '0', 4, 5, 6], dtype=object)

In [582]:
X_3

,floor,rooms,area,price,type,flatsize
0,6,2,51.000000,13400000.0,2.0,1
1,2,2,42.000000,17900000.0,2.0,1
2,5,2,42.000000,12500000.0,2.0,1
3,1,2,42.000000,10990000.0,2.0,1
4,4,2,51.000000,7450000.0,2.0,1
...,...,...,...,...,...,...
242416,3,1,29.000000,15950000.0,0.1,3
242417,10,2,89.599998,37500000.0,2.0,5
242418,1,1,34.400002,10500000.0,1.0,1
242419,12,3,105.400002,42000000.0,3.0,4


In [583]:
X_3.flatsize.dropna()

0         1
1         1
2         1
3         1
4         1
         ..
242416    3
242417    5
242418    1
242419    4
242420    1
Name: flatsize, Length: 228758, dtype: object

In [584]:
X_3['rooms'] = X_3['rooms'].astype('float')
X_3['floor'] = X_3['floor'].astype('float')
X_3['area'] = X_3['area'].astype('float')
X_3['type'] = X_3['type'].astype('float')
X_3['flatsize'] = X_3['flatsize'].astype('float')

C:\Users\smyko\AppData\Local\Temp\ipykernel_2016\307696765.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_3['rooms'] = X_3['rooms'].astype('float')
C:\Users\smyko\AppData\Local\Temp\ipykernel_2016\307696765.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_3['floor'] = X_3['floor'].astype('float')
C:\Users\smyko\AppData\Local\Temp\ipykernel_2016\307696765.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [585]:
X_3.dtypes

floor       float64
rooms       float64
area        float64
price       float32
type        float64
flatsize    float64
dtype: object

In [586]:
X_4 = X_3[['area', 'floor', 'rooms', 'type', 'flatsize']]
y = X_3['price']

In [587]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as MAE
from catboost import CatBoostRegressor
CatBoostRegressor().get_feature_importance

X_train, X_test, y_train, y_test = train_test_split(X_4, y, test_size=0.1, random_state=42)


model = CatBoostRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

Learning rate set to 0.095002
0:	learn: 18051675.8419029	total: 83.4ms	remaining: 1m 23s
1:	learn: 16984511.9625208	total: 154ms	remaining: 1m 16s
2:	learn: 16041561.9233033	total: 208ms	remaining: 1m 9s
3:	learn: 15222229.9563261	total: 257ms	remaining: 1m 4s
4:	learn: 14504993.4475265	total: 305ms	remaining: 1m
5:	learn: 13875996.7230377	total: 358ms	remaining: 59.3s
6:	learn: 13330084.2499304	total: 409ms	remaining: 58s
7:	learn: 12861377.1257040	total: 455ms	remaining: 56.4s
8:	learn: 12456040.8950955	total: 494ms	remaining: 54.4s
9:	learn: 12115411.8596770	total: 540ms	remaining: 53.5s
10:	learn: 11824701.1046248	total: 590ms	remaining: 53s
11:	learn: 11575699.7203166	total: 660ms	remaining: 54.4s
12:	learn: 11359446.0772595	total: 738ms	remaining: 56.1s
13:	learn: 11174741.7921224	total: 795ms	remaining: 56s
14:	learn: 11015445.2995131	total: 847ms	remaining: 55.6s
15:	learn: 10877878.2360931	total: 905ms	remaining: 55.6s
16:	learn: 10759451.2365109	total: 952ms	remaining: 55.1s


In [588]:
MAE1 = MAE(y_test, y_pred)
MAE1

5407566.004038935

In [589]:
X_train.head(4)

,area,floor,rooms,type,flatsize
145390,100.000000,3.0,4.0,4.0,0.0
237138,60.000000,9.0,3.0,3.0,0.0
23652,32.000000,3.0,1.0,1.0,1.0
146489,72.699997,2.0,3.0,3.0,1.0


In [590]:
flats_test = pd.read_excel('test_for_model.xlsx').set_index('flat_id')
flats_test = flats_test.fillna(0)
flats_test

,floor,rooms,area,flatsize,type
flat_id,,,,,
ТГ-МТГ02-5-5.9.8,9,1,16.3,1,0.1
ТГ-МТГ02-5-5.8.7,8,1,16.3,1,0.1
ТГ-МТГ02-5-5.7.7,7,1,16.3,1,0.1
ТГ-МТГ02-5-5.6.7,6,1,16.3,1,0.1
ТГ-МТГ02-5-5.5.7,5,1,16.3,1,0.1
...,...,...,...,...,...
КВМ-КВМ01-8-8.1.2,1,4,150.6,5,4.0
КВМ-КВМ01-2-2.1.2,1,3,139.7,6,3.0
КВМ-КВМ01-4-4.1.1,1,3,127.3,5,3.0


In [591]:
result=pd.DataFrame(model.predict(flats_test), index=flats_test.index, columns=['flatprice'])

In [592]:
result

,flatprice
flat_id,
ТГ-МТГ02-5-5.9.8,6.498282e+06
ТГ-МТГ02-5-5.8.7,6.571901e+06
ТГ-МТГ02-5-5.7.7,6.474677e+06
ТГ-МТГ02-5-5.6.7,6.380547e+06
ТГ-МТГ02-5-5.5.7,6.309122e+06
...,...
КВМ-КВМ01-8-8.1.2,6.637758e+07
КВМ-КВМ01-2-2.1.2,7.554379e+07
КВМ-КВМ01-4-4.1.1,3.912942e+07


In [593]:
result.dtypes

flatprice    float64
dtype: object

In [594]:
result['price'] = round(result['flatprice'], -4)

In [595]:
result

,flatprice,price
flat_id,,
ТГ-МТГ02-5-5.9.8,6.498282e+06,6500000.0
ТГ-МТГ02-5-5.8.7,6.571901e+06,6570000.0
ТГ-МТГ02-5-5.7.7,6.474677e+06,6470000.0
ТГ-МТГ02-5-5.6.7,6.380547e+06,6380000.0
ТГ-МТГ02-5-5.5.7,6.309122e+06,6310000.0
...,...,...
КВМ-КВМ01-8-8.1.2,6.637758e+07,66380000.0
КВМ-КВМ01-2-2.1.2,7.554379e+07,75540000.0
КВМ-КВМ01-4-4.1.1,3.912942e+07,39130000.0


In [596]:
result.to_excel('result.xlsx')